In [1]:
import numpy as np

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
import torch.utils.data as utils
from torchvision import datasets, transforms
from torch.utils.data.sampler import SubsetRandomSampler
import os
import platform
import pickle

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!rm -r DLAV-2024
!git clone https://github.com/vita-epfl/DLAV-2024.git
path = os.getcwd() + '/DLAV-2024/homeworks/hw2/test_batch'

In [33]:
# Set the variable to the location of the trained model
model_path = 'drive/MyDrive/Colab Notebooks/CNN_MiniVGG_deeperfully.ckpt'

In [34]:
# https://github.com/meng1994412/VGGNet_from_scratch/blob/master/pipeline/nn/conv/minivggnet.py
# I took inspiration from the MiniVggNet implementation in the above link

class MiniVGG(nn.Module):
    def __init__(self, n_input_channels=3, n_output=10):
        super(MiniVGG, self).__init__()
        self.conv1 = nn.Conv2d(n_input_channels, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 32, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(32)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.dropout1 = nn.Dropout(0.25)

        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(64)
        self.conv4 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(64)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.dropout2 = nn.Dropout(0.25)

        self.fc1 = nn.Linear(64 * 8 * 8, 512)
        self.bn5 = nn.BatchNorm1d(512)
        self.dropout3 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(512, 256)
        self.bn6 = nn.BatchNorm1d(256)
        self.fc3 = nn.Linear(256, n_output)



    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.pool1(F.relu(self.bn2(self.conv2(x))))
        x = self.dropout1(x)

        x = F.relu(self.bn3(self.conv3(x)))
        x = self.pool2(F.relu(self.bn4(self.conv4(x))))
        x = self.dropout2(x)

        x = x.view(-1, 64 * 8 * 8)
        x = F.relu(self.bn5(self.fc1(x)))
        x = self.dropout3(x)
        x = F.relu(self.bn6(self.fc2(x)))
        x = self.dropout3(x)
        x = self.fc3(x)
        return x

    def predict(self, x):
        logits = self.forward(x)
        return F.softmax(logits)


In [35]:
def predict_usingCNN(X):
    #########################################################################
    # TODO:                                                                 #
    # - Load your saved model                                               #
    # - Do the operation required to get the predictions                    #
    # - Return predictions in a numpy array                                 #
    # Note: For the predictions, you have to return the index of the max    #
    # value                                                                 #
    #########################################################################
    # Load the model
    #model = ConvNet()
    model = MiniVGG()
    model.load_state_dict(torch.load(model_path,map_location=torch.device('cpu')))
    model.eval()
    # Predict the output
    y_pred = model.predict(X)
    y_pred = torch.argmax(y_pred, dim=1).numpy()

    #########################################################################
    #                       END OF YOUR CODE                                #
    #########################################################################
    return y_pred


In [36]:
## Read DATA
def load_pickle(f):
    version = platform.python_version_tuple()
    if version[0] == '2':
        return  pickle.load(f)
    elif version[0] == '3':
        return  pickle.load(f, encoding='latin1')
    raise ValueError("invalid python version: {}".format(version))

def load_CIFAR_batch(filename):
  """ load single batch of cifar """
  with open(filename, 'rb') as f:
    datadict = load_pickle(f)
    X = datadict['data']
    Y = datadict['labels']
    X = X.reshape(10000, 3, 32, 32).astype("float")
    Y = np.array(Y)
    return X, Y
test_filename = path
X,Y = load_CIFAR_batch(test_filename)

In [37]:
# Data Manipulation
mean_pytorch = np.array([0.4914, 0.4822, 0.4465])
std_pytorch = np.array([0.2023, 0.1994, 0.2010])
X_pytorch = np.divide(np.subtract( X/255 , mean_pytorch[np.newaxis, :,np.newaxis,np.newaxis]), std_pytorch[np.newaxis, :,np.newaxis,np.newaxis])

# Run Prediction and Evaluation
prediction_cnn = predict_usingCNN(torch.from_numpy(X_pytorch).float())
acc_cnn = sum(prediction_cnn == Y)/len(X_pytorch)
print("CNN Accuracy= %f"%(acc_cnn))

<ipython-input-34-a75a06d89ae1>:49: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(logits)


CNN Accuracy= 0.851900
